## Getting the data

In this Jupyter notebook, I want to keep track of a few things:
1. Where we are sourcing our data.
2. The method by which we obtain data (e.g. scraping, using an API, etc.)
3. Some preliminary file organization.

Recall that we are trying to identify a relationship between the lobbying behavior and the individual stock trades of congresspeople. Let's start with the lobbying data.

Before doing anything though, I will keep here a code box for all of the packages and imported functions we use. This way, if you want to run a single code box in the notebook, you likely only have to run this one first.

In [3]:
import pandas as pd
import numpy as np

import requests
from requests.auth import HTTPBasicAuth
import json
from time import sleep

from crpapi import CRP
from API_Keys import get_opensecrets_key #These are for my API keys - you should register for your own and put them in your python installation under API_keys to make these imports work.
from API_Keys import get_LDA_key

/var/folders/ln/nt00cb4n18752q_3n96dk_2c0000gn/T/ipykernel_30601/2301786708.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Lobbying data

#### opensecrets.org

opensecrets.org is a non-profit watchdog organization that tries to keep track of money in politics. I think it is a good first place to look for finding out the different sectors of lobbying, and for determining broad-stroke impressions about money in politics. For instance - it has information on
1. Personal financial disclosures of congresspeople and estimates of their net worth.
2. Campaign contributions/ fundraising data.
3. Political ads by industries, either through 527s (issue advocacy groups) or PACs (often businesses, labor unions, or ideological interests)
4. __Domestic and foreign lobbying efforts, in the traditional sense__.
   
This last one is what we are most interested in, at least for an initial pass. 

NB: All the data opensecrets.org has is itself aggregated from different sources. In particular, data on lobbying efforts is all taken from disclosures from the office of the senate - we will look at this source next, as it is likely more granular in nature.

We will use the CRP API from opensecrets.org to access their data.
The API is easy to register for here: https://www.opensecrets.org/open-data/api. 


##### Using the python library

There is a python client library, due to Rob Remington, "opensecrets-crpapi" which allows us to interface with the CRP API through python. See: https://github.com/robrem/opensecrets-crpapi. This library is, however, 7 years old!
We can try to use this to download some data.

In [1]:
#pip install opensecrets-crpapi

Note: you may need to restart the kernel to use updated packages.


In [1]:
from crpapi import CRP
from API_Keys import get_opensecrets_key

crp=CRP(get_opensecrets_key()) #makes a CRP object

object_methods = [method_name for method_name in dir(crp) if callable(getattr(crp, method_name))]
print(dir(crp))



['BASE_URI', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'apikey', 'candidates', 'committees', 'fetch', 'http', 'indexp', 'orgs']


It looks like 'candidates', 'committees', 'fetch', 'http', 'indexp', 'orgs' are callable class methods in the library. 'candidates' is documented in the readme - it lets you get basic information on a legislator through their "CID".

In [2]:
crp.candidates.get('N00007360')

{'@attributes': {'cid': 'N00007360',
  'firstlast': 'Nancy Pelosi',
  'lastname': 'Pelosi',
  'party': 'D',
  'office': 'CA11',
  'gender': 'F',
  'first_elected': '1987',
  'exit_code': '0',
  'comments': '',
  'phone': '202-225-4965',
  'fax': '202-225-8259',
  'website': 'http://pelosi.house.gov',
  'webform': 'http://pelosi.house.gov/contact-me/email-me',
  'congress_office': '233 Cannon House Office Building',
  'bioguide_id': 'P000197',
  'votesmart_id': '26732',
  'feccandid': 'H8CA05035',
  'twitter_id': 'NancyPelosi',
  'youtube_url': 'https://youtube.com/nancypelosi',
  'facebook_id': 'NancyPelosi',
  'birthdate': '1940-03-26'}}

Calling candidates.industries(CID=,year=) will return the industries that have contributed to the candidate until that year.

In [9]:
crp.candidates.industries('N00007360',"2023")

[{'@attributes': {'industry_code': 'W06',
   'industry_name': 'Retired',
   'indivs': '5711313',
   'pacs': '0',
   'total': '5711313'}},
 {'@attributes': {'industry_code': 'H01',
   'industry_name': 'Health Professionals',
   'indivs': '361377',
   'pacs': '183500',
   'total': '544877'}},
 {'@attributes': {'industry_code': 'W04',
   'industry_name': 'Education',
   'indivs': '521707',
   'pacs': '0',
   'total': '521707'}},
 {'@attributes': {'industry_code': 'F07',
   'industry_name': 'Securities & Investment',
   'indivs': '420995',
   'pacs': '47500',
   'total': '468495'}},
 {'@attributes': {'industry_code': 'K01',
   'industry_name': 'Lawyers/Law Firms',
   'indivs': '402805',
   'pacs': '63500',
   'total': '466305'}},
 {'@attributes': {'industry_code': 'F10',
   'industry_name': 'Real Estate',
   'indivs': '336502',
   'pacs': '45000',
   'total': '381502'}},
 {'@attributes': {'industry_code': 'B02',
   'industry_name': 'TV/Movies/Music',
   'indivs': '326385',
   'pacs': '9000

In [11]:
crp.candidates.summary('N00007360',"2023")

{'cand_name': 'Pelosi, Nancy',
 'cid': 'N00007360',
 'cycle': '2024',
 'state': 'CA',
 'party': 'D',
 'chamber': 'H',
 'first_elected': '1987',
 'next_election': '2024',
 'total': '4903098.11',
 'spent': '4881103.22',
 'cash_on_hand': '3615723.65',
 'debt': '0',
 'origin': 'OpenSecrets',
 'source': 'https://www.opensecrets.org/members-of-congress/summary?cid=N00007360&cycle=2024',
 'last_updated': '02/14/2024'}

Certainly the Python library has limited functionality - we should interface with the opensecrets API more directly.

##### Using the CRP API directly

We do this using the requests package. The CRP API is well documented here: https://www.opensecrets.org/open-data/api-documentation.

In [6]:
from requests.auth import HTTPBasicAuth
import requests
import json
from time import sleep

In [4]:
auth= HTTPBasicAuth('apikey', get_opensecrets_key()) # Creates an authorization object that can be fed into requests when the API key is needed. Unfortunately, I don't know how to pass this object to actually feed the API my key.
CRP_url= "https://opensecrets.org/api/"
CRP_params ={}

Now, let's make some calls.

### Lobbying disclosures from the Office of the Senate (LDA)

Since lobbying data from opensecrets seems a bit coarse, we can try looking at the API offered by the Office of the Senate. 

Let's run through and grab the filings for lobbyists from various years. We first establish some basics.

In [9]:
from API_Keys import get_LDA_key

LDA_url="https://lda.senate.gov/api/v1/"
LDA_headers={"Authorization":"Token "+get_LDA_key()}

def ceil(a,b):
    return -(-a//b)

The ceiling division function defined above will be useful in determining the total number of pages our search provided (there are 25 entries available on each page).

We now grab all filings for 2024, saving them as jsons. There are 558 (visible below in the expression filings.json(["count"]) - this is the same when filings is pointing at any of the pages k).

In [67]:
i=0
LDA_params = {"method":"listFilings","filing_year": str(2024-i)}
filings=requests.get(LDA_url+"filings/", params=LDA_params, headers=LDA_headers)
m=ceil(filings.json()["count"],25)
for k in range(m):
    LDA_params = {"method":"listFilings","filing_year": str(2024-i),"page":str(k+1)}
    filings=requests.get(LDA_url+"filings/", params=LDA_params, headers=LDA_headers)
    with open("./LDA_data/Filings_2024/filings_"+str(2024-i)+"_p"+str(k+1)+".json", "w") as outfile:
        json.dump(filings.json(), outfile)

    

In [35]:
filings.json()["count"]

558

We import numpy and pandas as usual, so we can look at these files in DataFrame format.

In [5]:
import pandas as pd
import numpy as np

/var/folders/ln/nt00cb4n18752q_3n96dk_2c0000gn/T/ipykernel_9079/2162656668.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [11]:
filings_2024=pd.DataFrame()
for k in range(23): #23 was our old m 
    with open("LDA_data/Filings_2024/filings_2024_p"+str(k+1)+".json") as file:
        data = json.load(file)["results"]
        filings_2024=pd.concat([filings_2024,pd.DataFrame(data)], ignore_index=True)

filings_2024

,url,filing_uuid,filing_type,filing_type_display,filing_year,filing_period,filing_period_display,filing_document_url,filing_document_content_type,income,...,registrant_different_address,registrant_city,registrant_state,registrant_zip,registrant,client,lobbying_activities,conviction_disclosures,foreign_entities,affiliated_organizations
0,https://lda.senate.gov/api/v1/filings/7866327b...,7866327b-c892-4430-b9f0-1f0f679c58c6,RR,Registration,2024,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/7...,text/html,None,...,None,WASHINGTON,DC,20036,"{'id': 35707, 'url': 'https://lda.senate.gov/a...","{'id': 58116, 'url': 'https://lda.senate.gov/a...","[{'general_issue_code': 'DIS', 'general_issue_...",[],[],[]
1,https://lda.senate.gov/api/v1/filings/295a9fd5...,295a9fd5-5775-4a38-b8d3-8897973ab6cd,RR,Registration,2024,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/2...,text/html,None,...,None,Washington,DC,20005,"{'id': 321555, 'url': 'https://lda.senate.gov/...","{'id': 58117, 'url': 'https://lda.senate.gov/a...","[{'general_issue_code': 'NAT', 'general_issue_...",[],[],[]
2,https://lda.senate.gov/api/v1/filings/467e4a97...,467e4a97-6351-4902-8ffa-dd51632e156b,Q1,1st Quarter - Report,2024,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/4...,text/html,30000.00,...,False,Washington,DC,20002,"{'id': 401107792, 'url': 'https://lda.senate.g...","{'id': 56764, 'url': 'https://lda.senate.gov/a...","[{'general_issue_code': 'TEC', 'general_issue_...",[],[],[]
3,https://lda.senate.gov/api/v1/filings/d4c934f7...,d4c934f7-dab7-4600-95dc-e0f1ff05a8a7,Q1,1st Quarter - Report,2024,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/d...,text/html,15000.00,...,False,Washington,DC,20002,"{'id': 401107792, 'url': 'https://lda.senate.g...","{'id': 57213, 'url': 'https://lda.senate.gov/a...","[{'general_issue_code': 'INT', 'general_issue_...",[],[],[]
4,https://lda.senate.gov/api/v1/filings/f32a7add...,f32a7add-ee52-4ac8-b28a-7720917ae371,RR,Registration,2024,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/f...,text/html,None,...,None,Washington,DC,20006,"{'id': 401103955, 'url': 'https://lda.senate.g...","{'id': 58125, 'url': 'https://lda.senate.gov/a...","[{'general_issue_code': 'GOV', 'general_issue_...",[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,https://lda.senate.gov/api/v1/filings/4fdfde6f...,4fdfde6f-8080-4564-97b9-b774d0eae1d2,RR,Registration,2024,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/4...,text/html,None,...,None,Washington,DC,20006,"{'id': 24486, 'url': 'https://lda.senate.gov/a...","{'id': 58919, 'url': 'https://lda.senate.gov/a...","[{'general_issue_code': 'AUT', 'general_issue_...",[],[],[]
554,https://lda.senate.gov/api/v1/filings/ffad79e3...,ffad79e3-a11f-4f04-8d11-b5122ee26ee0,RR,Registration,2024,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/f...,text/html,None,...,None,Alexandria,VA,22301,"{'id': 401105227, 'url': 'https://lda.senate.g...","{'id': 58920, 'url': 'https://lda.senate.gov/a...","[{'general_issue_code': 'DEF', 'general_issue_...",[],[],[]
555,https://lda.senate.gov/api/v1/filings/88d130aa...,88d130aa-3358-4efa-b853-f12cecedec3f,RR,Registration,2024,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/8...,text/html,None,...,None,Washington,DC,20001,"{'id': 400711132, 'url': 'https://lda.senate.g...","{'id': 58921, 'url': 'https://lda.senate.gov/a...","[{'general_issue_code': 'FUE', 'general_issue_...",[],[],[]
556,https://lda.senate.gov/api/v1/filings/d9368a0d...,d9368a0d-2771-4ab9-b0b3-800861704526,RA,Registration - Amendment,2024,first_quarter,1st Quarter (Jan 1 - Mar 31),https://lda.senate.gov/filings/public/filing/d...,text/html,None,...,None,ROCKVI

Let me save this as one file for future access.

In [14]:
filings_2024.to_json('./LDA_data/Filings_2024/filings_2024_total.json', orient='records', lines=True)


Looks good. Now let's grab the 2023 data - there should be a lot more compared to the 2024 data, given that we are only in February 2024 right now.

In [76]:
LDA_params = {"method":"listFilings","filing_year": str(2024-1)}
filings=requests.get(LDA_url+"filings/", params=LDA_params, headers=LDA_headers)
m=ceil(filings.json()["count"],25)

In [16]:
print(filings.json()["count"],m)

93683 3748


Oh dear, there are 93,683 filings for 2023. (Rem: This number changes later - the office of the senate released some late filings.) That means we expect 3,748 files when we download.

In [77]:
i=1
LDA_params = {"method":"listFilings","filing_year": str(2024-i)}
filings=requests.get(LDA_url+"filings/", params=LDA_params, headers=LDA_headers)
m=ceil(filings.json()["count"],25)
for k in range(m):
    LDA_params = {"method":"listFilings","filing_year": str(2024-i),"page":str(k+1)}
    filings=requests.get(LDA_url+"filings/", params=LDA_params, headers=LDA_headers)
    with open("./LDA_data/Filings_2023/filings_"+str(2024-i)+"_p"+str(k+1)+".json", "w") as outfile:
        json.dump(filings.json(), outfile)

It took a little over 2 hours of this code running to download the 2023 filings (but it did it!). In the future, we may wish to parallelize the procedure a bit. In the above we were making roughly 28 calls a minute, while the API throttles us at 200 calls a minute. So we could easily speed up things by a factor of 6 or 7 by breaking up and running many calls simultaneously.

In any case, we definitely don't want to make a pandas dataframe with all 93,638 rows. Instead, let's first work with a random selection of 50 of the 3,749 files.

In [11]:
np.random.seed(691)
ind=list(np.random.randint(1,3749,50)) #choosing a list of 50 random files to pull data on
filings_2023=pd.DataFrame()
for k in ind:
    with open("LDA_data/Filings_2023/filings_2023_p"+str(k+1)+".json") as file:
        data = json.load(file)["results"]
        filings_2023=pd.concat([filings_2023,pd.DataFrame(data)], ignore_index=True)


In [79]:
filings_2023.keys()

Index(['url', 'filing_uuid', 'filing_type', 'filing_type_display',
       'filing_year', 'filing_period', 'filing_period_display',
       'filing_document_url', 'filing_document_content_type', 'income',
       'expenses', 'expenses_method', 'expenses_method_display',
       'posted_by_name', 'dt_posted', 'termination_date', 'registrant_country',
       'registrant_ppb_country', 'registrant_address_1',
       'registrant_address_2', 'registrant_different_address',
       'registrant_city', 'registrant_state', 'registrant_zip', 'registrant',
       'client', 'lobbying_activities', 'conviction_disclosures',
       'foreign_entities', 'affiliated_organizations'],
      dtype='object')

In [10]:
df=pd.DataFrame({"Year": [2024-i for i in range(12)], "Total_filings": [0,0,0,0,0,0,0,0,0,0,0,0], "Total_pages": [0,0,0,0,0,0,0,0,0,0,0,0]})

for i in range(12):
    LDA_params = {"method":"listFilings","filing_year": str(2024-i)}
    filings=requests.get(LDA_url+"filings/", params=LDA_params, headers=LDA_headers)
    df.loc[df.Year==2024-i, "Total_filings"]=filings.json()["count"]
    df.loc[df.Year==2024-i, "Total_pages"]=ceil(filings.json()["count"],25)

df

,Year,Total_filings,Total_pages
0,2024,658,27
1,2023,93774,3751
2,2022,93271,3731
3,2021,90620,3625
4,2020,83991,3360
5,2019,79951,3199
6,2018,78401,3137
7,2017,77206,3089
8,2016,72184,2888
9,2015,73902,2957


In [26]:
with open("./LDA_data/filings_info.csv", "w") as outfile:
    df.to_csv(outfile, index=False)

The following (commented out) block of code will download all returns back to 2013. While running it overnight, the LDA API shut down for maintenance so I had to restart it a few times with some changes. Still, the original code is here.

In [21]:
# for i in range(12):
#     LDA_params = {"method":"listFilings","filing_year": str(2024-i)}
#     filings=requests.get(LDA_url+"filings/", params=LDA_params, headers=LDA_headers)
#     m=ceil(filings.json()["count"],25)
#     for k in range(m):
#         LDA_params = {"method":"listFilings","filing_year": str(2024-i),"page":str(k+1)}
#         filings=requests.get(LDA_url+"filings/", params=LDA_params, headers=LDA_headers)
#         with open("./LDA_data/Filings_"+str(2024-i)+"/filings_"+str(2024-i)+"_p"+str(k+1)+".json", "w") as outfile:
#             json.dump(filings.json(), outfile)

OK, let's sort these files a bit. There are a number of filing types - let's sort by these.

In [42]:
filing_types=["RR","RA","Q1","Q1Y","1T","1TY","1A","1AY","1@","1@Y","Q2","Q2Y","2T","2TY","2A","2AY","2@","2@Y","Q3","Q3Y","3T","3TY","3A","3AY","3@","3@Y","Q4","Q4Y","4T","4TY","4A","4AY","4@","4@Y","MM","MMY","MT","MTY","MA","MAY","M@","M@Y","YY","YYY","YT","YTY","YA","YAY","Y@","Y@Y"]

And we loop through these and sort our data into different files. Again, I have commented out the code since I don't want to accidentally re-run it now that I have done so once.

In [87]:
# with open("./LDA_data/filings_info.csv","r") as index_file:
#     index_df=pd.read_csv(index_file)
# for i in range(1,12):
#     m=index_df["Total_pages"].iloc[i]
#     for filing_type in filing_types:
#         collection=pd.DataFrame() #this is a temporary dataframe we will fill with appropriate filings
#         for k in range(m):
#             with open("./LDA_data/Filings_"+str(2024-i)+"/filings_"+str(2024-i)+"_p"+str(k+1)+".json", "r") as file:
#                 data = pd.DataFrame(json.load(file)["results"])
#                 data_filing_type=data.loc[data.filing_type==filing_type]
#                 collection=pd.concat([collection,data_filing_type], ignore_index=True) #this fills collection with all filings of a particular filing type for the year
#         with open("./LDA_data/Filings_"+str(2024-i)+"/filings_"+str(2024-i)+"_"+filing_type+".csv", "w+") as outfile:
#                 collection.to_csv(outfile, index=False)




Having now produced these sorted filings, I saved the original downloaded files elsewhere. The original files can be found on Google Drive, as can these sorted ones.

What do these different types of filing codes mean?

|Code | Meaning| 
------|-------
|R | Registration |
|A |  Amendment |
|Q | Quarter |
|Y | No activity |
|T | Termination (lobbying for a client has finished) |
|@ | Termination amendment |

The other codes are not relevant - there are no filings from 2013-2023 with codes starting with an M or a Y.

### Trading data

We sourced some trading data from QuiverQuant. We may need more. Let's take a quick look at it.

In [4]:
stocks=pd.read_excel("./trading_data/congress-trading-all.xlsx")

In [5]:
stocks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46363 entries, 0 to 46362
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Ticker              46363 non-null  object 
 1   TickerType          33426 non-null  object 
 2   Company             46362 non-null  object 
 3   Traded              46363 non-null  object 
 4   Transaction         46363 non-null  object 
 5   Trade_Size_USD      46363 non-null  object 
 6   Status              46273 non-null  object 
 7   Subholding          27336 non-null  object 
 8   Description         4324 non-null   object 
 9   Name                46363 non-null  object 
 10  Filed               46363 non-null  object 
 11  Party               46363 non-null  object 
 12  District            36310 non-null  object 
 13  Chamber             46363 non-null  object 
 14  Comments            813 non-null    object 
 15  Quiver_Upload_Time  46363 non-null  object 
 16  exce

In [14]:
stocks.sample(50)

,Ticker,TickerType,Company,Traded,Transaction,Trade_Size_USD,Status,Subholding,Description,Name,Filed,Party,District,Chamber,Comments,Quiver_Upload_Time,excess_return,State,last_modified
31214,EPD,NaN,ENTERPRISE PRODUCTS PARTNERS L.P.,2017-11-08,Purchase,"$1,001 - $15,000",NEW,NaN,NaN,Virginia Foxx,2017-12-12,R,NC05,House,NaN,2020-07-26,-84.852477,North Carolina,2023-11-16
7236,BBH,ST,VANECK BIOTECH ETF,2022-03-04,Sale,"$50,001 - $100,000",NEW,MERRILL LYNCH- ADVISOR DISCRETION ACCOUNT,NaN,Thomas Suozzi,2022-05-20,D,NY03,House,NaN,2022-05-23,-8.065386,New York,2023-11-16
44980,SPG,NaN,"SIMON PROPERTY GROUP, INC.",2014-04-15,Purchase,"$1,001 - $15,000",NEW,JUDITH S. CLARK IRREVOCABLE TRUST,NaN,Katherine M. Clark,2014-12-19,D,MA05,House,NaN,2020-07-26,-188.005575,Massachusetts,2023-11-16
18823,UL,ST,UNILEVER PLC,2020-03-03,Purchase,"$1,001 - $15,000",NEW,NaN,PART OF MY SPOUSE'S RETIREMENT PORTFOLIO.,Earl Blumenauer,2020-04-16,D,OR03,House,NaN,2020-07-26,-80.106450,Oregon,2023-11-16
14732,GE,ST,GENERAL ELECTRIC COMPANY,2020-08-24,Purchase,"$1,001 - $15,000",NEW,NaN,NaN,Austin Scott,2020-09-20,R,GA08,House,NaN,2020-09-27,142.460711,Georgia,2023-11-16
36986,DBSDY,NaN,DBS GROUP HOLDINGS LTD ORD AMERICAN DEPOSITARY...,2016-06-06,Purchase,"$1,001 - $15,000",NEW,THOMAS C MACARTHUR AND DEBORAH A MACARTHUR,NaN,Thomas Macarthur,2016-07-08,R,NJ03,House,NaN,2020-07-26,NaN,New Jersey,2023-11-16
9166,PSXP,ST,PHILLIPS 66 PARTNERS LP COMMON UNITS REPRESENT...,2021-09-02,Purchase,"$250,001 - $500,000",NEW,AXA RETIREMENT ACCOUNT,NaN,Mark Dr Green,2021-09-23,R,TN07,House,NaN,2021-09-24,23.641495,Tennessee,2023-11-16
3355,CSII,Stock,"Cardiovascular Systems, Inc. - Common Stock",2023-04-05,Sale (Full),"$50,001 - $100,000",New,NaN,NaN,"Smith, Tina",2023-04-19,D,NaN,Senate,Underlying asset of Truchas Capital 401k Plan,2023-04-19,-0.592424,Minnesota,2023-08-28
36136,WEX,NaN,WEX INC. COMMON STOCK,2016-09-12,Sale,"$1,001 - $15,000",NEW,SP M81 (MERRILL LYNCH),NaN,K. Michael Conaway,2016-10-12,R,TX11,House,NaN,2020-07-26,-24.153889,Texas,2023-11-16
31253,VB,NaN,VANGUARD SMALL-CAP ETF,2017-11-07,Sale,"$15,001 - $50,000",NEW,NaN,NaN,Barbara J Comstock,2017-11-08,R,VA10,House,NaN,2020-07-26,-42.813224,Virginia,2023-11-16


In [7]:
stocks.Filed.min()

'2014-01-03'

In [13]:
stocks.TickerType.value_counts()

TickerType
ST                  24088
Stock                7701
OP                    567
Stock Option          430
Other Securities      349
OT                    102
CS                     66
GS                     58
AB                     19
ET                     13
HN                      9
Corporate Bond          9
PS                      4
Cryptocurrency          4
OI                      3
OL                      3
SA                      1
Name: count, dtype: int64